# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></div><div class="lev1 toc-item"><a href="#Paths" data-toc-modified-id="Paths-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Paths</a></div><div class="lev1 toc-item"><a href="#Group-Model" data-toc-modified-id="Group-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Group Model</a></div><div class="lev2 toc-item"><a href="#Load-Data" data-toc-modified-id="Load-Data-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load Data</a></div><div class="lev2 toc-item"><a href="#Create-Regressor-Columns" data-toc-modified-id="Create-Regressor-Columns-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Create Regressor Columns</a></div><div class="lev2 toc-item"><a href="#Separate-Regressors" data-toc-modified-id="Separate-Regressors-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Separate Regressors</a></div><div class="lev2 toc-item"><a href="#Convert-Choices" data-toc-modified-id="Convert-Choices-34"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Convert Choices</a></div><div class="lev2 toc-item"><a href="#Confidence" data-toc-modified-id="Confidence-35"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Confidence</a></div><div class="lev2 toc-item"><a href="#Fast-RTs" data-toc-modified-id="Fast-RTs-36"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Fast RTs</a></div><div class="lev1 toc-item"><a href="#Regressor" data-toc-modified-id="Regressor-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Regressor</a></div><div class="lev2 toc-item"><a href="#Model" data-toc-modified-id="Model-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Model</a></div><div class="lev2 toc-item"><a href="#Save-Posterior-Plots" data-toc-modified-id="Save-Posterior-Plots-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Save Posterior Plots</a></div><div class="lev1 toc-item"><a href="#PPC" data-toc-modified-id="PPC-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>PPC</a></div>

# Import Libraries

In [2]:
###HDDM SCRIPT: JOHN CLITHERO###
###UPDATED: 23 March 2016###

### UPDATED by Daniel J Wilson

import hddm
import pandas as pd
import numpy as np
import os,sys
import matplotlib.pyplot as plt

/Users/djw/anaconda/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


# Paths

In [46]:
###paths###
date_var = '170906' 
exp_dir = '/Users/djw/Dropbox/PROGRAMMING/*NEURO/hDDM_Clithero/Code' #make sure this is correct
data_dir = os.path.join(exp_dir, 'Data/CSV_data/')
analysis_dir = os.path.join(exp_dir,'Analysis','HDDM')
date_analysis_dir = os.path.join(analysis_dir,date_var)
if os.path.isdir(date_analysis_dir) == False:
    os.mkdir(date_analysis_dir)
group_analysis_dir = os.path.join(date_analysis_dir,'Group')
if os.path.isdir(group_analysis_dir) == False:
    os.mkdir(group_analysis_dir)
group_reg5_analysis_dir = os.path.join(group_analysis_dir,'GroupReg5')
if os.path.isdir(group_reg5_analysis_dir) == False:
    os.mkdir(group_reg5_analysis_dir)
group_reg5_posterior_analysis_dir = os.path.join(group_reg5_analysis_dir,'Posteriors')
if os.path.isdir(group_reg5_posterior_analysis_dir) == False:
    os.mkdir(group_reg5_posterior_analysis_dir)

# Group Model

## Load Data

In [47]:
###group model###
num_samples = 500  # normally 11000
num_reg_samples = 500  # normally 11000
num_burn = 50  # normally 500
num_subs = 38 #full sample
#group_csv = 'foodother_Xsub_n' + str(num_subs) + '_hddm.csv'
group_csv = 'SubjectPlusOtherRatings.csv'
group_csv_path = os.path.join(data_dir,group_csv)
group_data = hddm.load_csv(group_csv_path)

# check num_subs later on
# change group_csv name

In [48]:
group_data.head()

,﻿Subject,Trial,Instruction,Food,TasteRating,TasteRatingRT,HealthRating,HealthRatingRT,Resp,RT,...,NonRated2,MostSimRating,MostDiffRating,ChoseHealthiest,ChoseTastiest,Block,DiffPrevBlock,SimPrevBlock,SimActual,DiffActual
0,1,1,Self,FrootLoops.bmp,3,1.2822,1,1.4234,3,1.2752,...,3.0,3.0,1.0,False,True,0,NaN,NaN,3.0,1.0
1,1,2,Self,nesquick.bmp,3,5.3458,3,1.1263,3,1.2006,...,3.0,3.0,3.0,True,True,0,NaN,NaN,3.0,1.0
2,1,3,Self,combos.bmp,1,2.2474,1,1.1789,1,1.4586,...,1.0,1.0,1.0,True,True,0,NaN,NaN,3.0,NaN
3,1,4,Self,Fritos.bmp,3,1.1329,1,2.2720,3,0.9781,...,3.0,3.0,1.0,False,True,0,NaN,NaN,1.0,1.0
4,1,5,Self,kiwi.bmp,4,1.0342,4,0.8464,4,1.0347,...,4.0,4.0,4.0,True,True,0,NaN,NaN,4.0,3.0


In [49]:
group_data = group_data.rename(columns={'Subject': 'subj_idx', 'Resp': 'response', 'RT': 'rt', })

## Create Regressor Columns

In [50]:
group_data['type'] = 0
group_data['type'][group_data['Instruction'] == 'Self'] = 1
group_data['type'][group_data['Instruction'] == 'Sim'] = 2
group_data['type'][group_data['Instruction'] == 'Diff'] = 3

/Users/djw/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/djw/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/djw/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Separate Regressors

In [51]:
##separate regressors for rating on trial type##
self_index = group_data.type == 1
sim_index = group_data.type == 2
diff_index = group_data.type == 3
group_data['TasteSelf'] = 0
group_data['TasteSim'] = 0
group_data['TasteDiff'] = 0
group_data['HealthSelf'] = 0
group_data['HealthSim'] = 0
group_data['HealthDiff'] = 0
group_data.loc[self_index,'TasteSelf'] = group_data.loc[self_index,'TasteRating']
group_data.loc[self_index,'HealthSelf'] = group_data.loc[self_index,'HealthRating']
group_data.loc[sim_index,'TasteSim'] = group_data.loc[sim_index,'TasteRating']
group_data.loc[sim_index,'HealthSim'] = group_data.loc[sim_index,'HealthRating']
group_data.loc[diff_index,'TasteDiff'] = group_data.loc[diff_index,'TasteRating']
group_data.loc[diff_index,'HealthDiff'] = group_data.loc[diff_index,'HealthRating']

In [52]:
group_data.head(20)

,﻿Subject,Trial,Instruction,Food,TasteRating,TasteRatingRT,HealthRating,HealthRatingRT,response,rt,...,SimPrevBlock,SimActual,DiffActual,type,TasteSelf,TasteSim,TasteDiff,HealthSelf,HealthSim,HealthDiff
0,1,1,Self,FrootLoops.bmp,3,1.2822,1,1.4234,3,1.2752,...,NaN,3.0,1.0,1,3,0,0,1,0,0
1,1,2,Self,nesquick.bmp,3,5.3458,3,1.1263,3,1.2006,...,NaN,3.0,1.0,1,3,0,0,3,0,0
2,1,3,Self,combos.bmp,1,2.2474,1,1.1789,1,1.4586,...,NaN,3.0,NaN,1,1,0,0,1,0,0
3,1,4,Self,Fritos.bmp,3,1.1329,1,2.2720,3,0.9781,...,NaN,1.0,1.0,1,3,0,0,1,0,0
4,1,5,Self,kiwi.bmp,4,1.0342,4,0.8464,4,1.0347,...,NaN,4.0,3.0,1,4,0,0,4,0,0
5,1,6,Self,mixedyogurt.bmp,3,4.1097,3,1.2895,3,1.1601,...,NaN,4.0,NaN,1,3,0,0,3,0,0
6,1,7,Self,plum.bmp,1,4.5806,3,1.4063,2,2.2826,...,NaN,1.0,1.0,1,1,0,0,3,0,0
7,1,8,Self,AppleJacks.bmp,3,1.1129,2,1.6787,3,0.8807,...,NaN,3.0,1.0,1,3,0,0,2,0,0
8,1,9,Self,chipmuffins.bmp,3,8.2699,2,1.1065,2,1.2326,...,NaN,4.0,1.0,1,3,0,0,2,0,0
9,1,10,Self,Dots_one.bmp,2,1.2425,1,1.3015,2,1.1462,...,NaN,3.0,1.0,1,2,0,0,1,0,0


## Convert Choices

In [53]:
##convert other choices to affect self choices##
group_data['SimOnSelf'] = group_data['SimRating'] - 2.5 #make negative for no choices
group_data['DiffOnSelf'] = group_data['SimRating'] - 2.5 #make negative for no choices
group_data.loc[sim_index,'SimOnSelf'] = 0 #make zero on all non-self trials
group_data.loc[diff_index,'SimOnSelf'] = 0 #make zero on all non-self trials
group_data.loc[sim_index,'DiffOnSelf'] = 0 #make zero on all non-self trials
group_data.loc[diff_index,'DiffOnSelf'] = 0 #make zero on all non-self trials

## Confidence

In [54]:
#confidence#
group_data['confidence'] = 0
yes_index = group_data.response == 4
no_index = group_data.response == 1 
group_data.loc[yes_index,'confidence'] = 1 #more likely to say yes
group_data.loc[no_index,'confidence'] = -1 #more likely to say no
group_data['confidenceSelf'] = 0
group_data['confidenceSim'] = 0
group_data['confidenceDiff'] = 0
group_data.loc[self_index,'confidenceSelf'] = group_data.loc[self_index,'confidence']
group_data.loc[sim_index,'confidenceSim'] = group_data.loc[sim_index,'confidence']
group_data.loc[diff_index,'confidenceDiff'] = group_data.loc[diff_index,'confidence']

## Fast RTs

In [55]:
###drop RT that are too fast###
#need to do this last since other params require history#
rt_thresh_lb = 0.200 #200 ms lower threshold
group_data = group_data[group_data['rt'] > rt_thresh_lb]
group_data = group_data.reset_index() #reset index

# Regressor

## Model

In [56]:
##reg5##
reg_model_num = 5
v_reg = 'v ~ TasteSelf + HealthSelf + TasteSim + HealthSim + TasteDiff + HealthDiff'
a_reg = 'a ~ C(type)'
reg = [v_reg,a_reg]
model_group_reg5 = hddm.HDDMRegressor(group_data,reg,include='z',informative=False,keep_regressor_trace=True) #use patsy
model_group_reg5_traces = os.path.join(group_reg5_analysis_dir,'traces_reg5.db')
model_group_reg5.sample(num_reg_samples,burn=num_burn,dbname=model_group_reg5_traces,db='pickle')
#print#
model_group_reg5_name = 'stats_print_all_model_group_n' + str(num_subs) + '_reg' + str(reg_model_num) + '.csv'
model_group_reg5_out_path = os.path.join(group_reg5_analysis_dir,model_group_reg5_name)
model_group_reg5.print_stats(fname=model_group_reg5_out_path)
#stats#
model_group_reg5_stats = model_group_reg5.gen_stats()
model_group_reg5_stats_name = 'stats_all_group_n' + str(num_subs) + '_reg' + str(reg_model_num) + '_stats.csv'
model_group_reg5_stats_out_path = os.path.join(group_reg5_analysis_dir,model_group_reg5_stats_name)
model_group_reg5_stats.to_csv(model_group_reg5_stats_out_path)
dic_reg5 = model_group_reg5.dic
#save reg5#
model_group_reg5_sname = 'model_group_reg' + str(reg_model_num) + '_n' + str(num_subs)
model_group_reg5_path = os.path.join(group_reg5_analysis_dir,model_group_reg5_sname)
model_group_reg5.save(model_group_reg5_path)

Adding these covariates:
['v_Intercept', 'v_TasteSelf', 'v_HealthSelf', 'v_TasteSim', 'v_HealthSim', 'v_TasteDiff', 'v_HealthDiff']
Adding these covariates:
['a_Intercept', 'a_C(type)[T.2]', 'a_C(type)[T.3]']
 [-----------------100%-----------------] 501 of 500 complete in 702.7 secWARNING: Will not save custom link functions.


## Save Posterior Plots

In [60]:
##save posterior plots for reg5##
model_group_reg5.plot_posteriors(save=True,path=group_reg5_posterior_analysis_dir)
plt.close('all')

Plotting t
Plotting z
Plotting v_Intercept
Plotting v_TasteSelf
Plotting v_HealthSelf
Plotting v_TasteSim
Plotting v_HealthSim
Plotting v_TasteDiff
Plotting v_HealthDiff


/Users/djw/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Plotting a_Intercept
Plotting a_C(type)[T.2]
Plotting a_C(type)[T.3]


# PPC

In [62]:
group_reg_ppc_analysis_dir = os.path.join(group_reg5_analysis_dir,'PPC')
if os.path.isdir(group_reg_ppc_analysis_dir) == False:
    os.mkdir(group_reg_ppc_analysis_dir)
group_model = model_group_reg5 #to keep it simple below
##generate ppc data##
##probably takes a while to generate for large number of samples##
num_samples = 100
ppc_data = hddm.utils.post_pred_gen(group_model,samples= num_samples)
ppc_data_name = date_var + '_ppc_data_model_group_reg' + str(reg_model_num) + '_n' + str(num_subs) + '_ns' + str(num_samples) + '.csv'
ppc_data_path = os.path.join(group_reg_ppc_analysis_dir,ppc_data_name)
ppc_data.to_csv(ppc_data_path)
##ppc_load = pd.read_csv(ppc_data_path,index_col=[0,1,2])
#ppc stats#
ppc_compare = hddm.utils.post_pred_stats(group_data, ppc_data)
#save to csv#
ppc_compare_name = date_var + '_ppc_stats_all_hddm_group_model_reg' + str(reg_model_num) + '_n' + str(num_subs) + '_ns' + str(num_samples) + '.csv'
ppc_compare_path = os.path.join(group_reg_ppc_analysis_dir,ppc_compare_name)
ppc_compare.to_csv(ppc_compare_path) #to read use index_col=[0] in read_csv
#save separate summary to csv#
ppc_mse_choice = ppc_compare.loc['accuracy']['MSE']
ppc_mse_rt_ub = ppc_compare.loc['mean_ub']['MSE'] #upper barrier
ppc_mse_rt_lb = ppc_compare.loc['mean_lb']['MSE'] #lower barrier (negative RT)
ppc_mh_choice = ppc_compare.loc['accuracy']['mahalanobis']
ppc_mh_rt_ub = ppc_compare.loc['mean_ub']['mahalanobis'] #upper barrier
ppc_mh_rt_lb = ppc_compare.loc['mean_lb']['mahalanobis'] #lower barrier (negative RT)
mse_array = np.array([ppc_mse_choice,ppc_mse_rt_ub,ppc_mse_rt_lb])
mh_array = np.array([ppc_mh_choice,ppc_mh_rt_ub,ppc_mh_rt_lb])
index_array = ['accuracy','mean_ub','mean_lb']
df_ppc_stats = pd.DataFrame(mse_array)
df_ppc_stats = df_ppc_stats.rename(columns = {0:'MSE'})
df_ppc_stats['mahalanobis'] = mh_array
df_ppc_stats.index = index_array
df_ppc_stats.index.name = 'stat'
df_ppc_stats_name = date_var + '_ppc_stats_summary_hddm_group_model_reg' + str(reg_model_num) + '_n' + str(num_subs) + '_ns' + str(num_samples) + '.csv'
df_ppc_stats_path = os.path.join(group_reg_ppc_analysis_dir,df_ppc_stats_name)
df_ppc_stats.to_csv(df_ppc_stats_path) #to read use index_col=[0] in read_csv

 [--------------------------------------------------------------------------400%--------------------------------------------------------------------------] 4 of 1 complete in 21047.3 sec

KeyError: 'response'

In [ ]:
5+5